# Seeded CaImAn using AGOnIA detection  
We'll use the boxes found with AGOnIA during the segmentation to seed the CaImAn algorithm and compare temporal traces

In [ ]:
import pickle

import bokeh.plotting as bpl
import logging
import matplotlib.pyplot as plt
import numpy as np
import cv2

from glob import glob

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.WARNING)

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import holoviews as hv
bpl.output_notebook()
hv.notebook_extension('bokeh')

import os.path

First we select the data to be used, which is a movie that has already been motion corrected by Luca. 

In [ ]:
data_path = '/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman' 
data_name = '501271265_5000'
fnames = [os.path.join(data_path,data_name + '.tif')]

Setup parameteres for detection. This step is very important, because the detection highly depends on this parameters. 
We should optimize this part to improve CaImAn detection and make a fair comparison.

In [ ]:
# dataset dependent parameters
fr = 1/0.758758526502838    # imaging rate in frames per second (data obtained from file ....)
decay_time = 0.65           # length of a typical transient in seconds (this is the value given by Marco Brondi, which is different from the one in the original notebook)

# motion correction parameters
strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system
gnb = 2                     # number of global background components
merge_thr = 0.85            # merging threshold, max correlation allowed
rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6             # amount of overlap between the patches in pixels
K = 4                       # number of components per patch
gSig = [8, 8]             # expected half size of neurons in pixels
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.85             # space correlation threshold for accepting a component
cnn_thr = 0.99              # threshold for CNN based classifier
cnn_lowest = 0.1            # neurons with cnn probability lower than this value are rejected

opts_dict = {'fnames': fnames,
            'fr': fr,
            'decay_time': decay_time,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'merge_thr': merge_thr, 
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': True,
            'min_cnn_thr': cnn_thr,
            'cnn_lowest': cnn_lowest}

opts = params.CNMFParams(params_dict=opts_dict)

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

## Motion Correction
First we create a motion correction object with the parameters specified. Note that the file is not loaded in memory
OBS: the movie we are using has already been motion corrected but in a different way. With this mc there are regions in the edge that are black at some points in time. 
This includes an extra problem for CAIMAN, we will perform motion correction on top of the previous motion correction for 2 reasons: first it makes the rest of the analysis easier, the creaton of the mmap is not trivial if its not created for the mc. And second because it can't hurt, it would just improve the detection. 

In [ ]:
# first we create a motion correction object with the parameters specified
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
# note that the file is not loaded in memory

In [ ]:
%%capture
#%% Run piecewise-rigid motion correction using NoRMCorre
mc.motion_correct(save_movie=True)
m_els = cm.load(mc.fname_tot_els)
border_to_0 = 0 if mc.border_nan is 'copy' else mc.border_to_0 
    # maximum shift to be used for trimming against NaNs

In [ ]:
#%% MEMORY MAPPING
# memory map the file in order 'C'
fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=border_to_0, dview=dview) # exclude borders

# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)

If the memmap and motion correction have already been done

In [ ]:
fname_new = '/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/memmap__d1_512_d2_512_d3_1_order_C_frames_5000_.mmap'
# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)

In [ ]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

## Build the seeds with AGONIA boxes
Up to here we used the code for the standard pipeline notebook. At this point we have motion corrected the movie and created the mmaps. 
We now want to create the masks for the seeded implementation using the boxes detected by agonia. 

In [ ]:
# Load the boxes in the pickle file into the ROIs arrays.  
with open('/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/501271265_boxes.pkl','rb') as f:
    cajas = pickle.load(f)
    f.close()
    
Ain = np.zeros((512*512,cajas.shape[0]),dtype=bool)
for i in range(len(cajas)):
    frame = np.zeros((512,512))
    frame[cajas[i,0].astype('int'):cajas[i,2].astype('int'),cajas[i,1].astype('int'):cajas[i,3].astype('int')]=1
    Ain[:,i] = frame.reshape(512*512).astype(bool)
Ain.shape

In [ ]:
crd = nb_plot_contour(images.mean(axis=0), Ain.astype('float32'), images.shape[1], images.shape[2], thr=0.99)

## Run CaImAn batch (CNMF) seeded with the set of the binary masks

In [ ]:
# dataset dependent parameters

rf = None                   # half-size of the patches in pixels. Should be `None` when seeded CNMF is used.
only_init = False           # has to be `False` when seeded CNMF is used
gSig = (8, 8)               # expected half size of neurons in pixels, very important for proper component detection

# params object
opts_dict = {'fnames': fnames,
            'decay_time': 0.65,
            'p': 1,
            'nb': 2,
            'rf': rf,
            'only_init': only_init,
            'gSig': gSig,
            'ssub': 1,
            'tsub': 1,
            'merge_thr': 0.85}

opts.change_params(opts_dict);

In [ ]:
cnm_seeded = cnmf.CNMF(n_processes, params=opts, dview=dview, Ain=Ain[:,:10])
cnm_seeded.fit(images)

In [ ]:
CI = cm.local_correlations(images.transpose(1,2,0))     #  correlation image
CI[np.isnan(CI)] = 0
cnm_seeded.estimates.plot_contours_nb(img=CI)

In [ ]:
cnm_seeded.save(os.path.join(data_path,data_name + '_analysis_results.hdf5'))

In [ ]:
cnm_seeded = cnmf.load_CNMF('/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/501271265_5000_analysis_results.hdf5')

In [ ]:
dims = (512,512)

In [ ]:
images.shape[1:]

In [ ]:
denoised = cm.movie(cnm2.estimates.A.dot(cnm2.estimates.C) + \
                    cnm2.estimates.b.dot(cnm2.estimates.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])

In [ ]:
denoised.resize(1, 1, 0.2).play(q_max=99.5, fr=1/0.758758526502838, magnification=2)

In [ ]:
cnm2.estimates.select_components(use_object=True)

In [ ]:
cnm_seeded.estimates.evaluate_components(images, cnm_seeded.params, dview=dview)

In [ ]:
cnm_seeded.estimates.plot_contours_nb(images.mean(axis=0), idx=cnm_seeded.estimates.idx_components)

In [ ]:
cnm_seeded.estimates.nb_view_components(img=images.mean(axis=0), idx=cnm_seeded.estimates.idx_components_bad)